In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate() 
sqlContext = SQLContext(sc)

In [2]:
from imageToPixels import *

In [3]:
folderName = '/Users/davidkes/classes/697/Project/wiki_crop/' 
fileNames = glob.glob(folderName +'*_resized/*.jpg')
# only test 20 files for now
NumberOfFileToBetested = 20

x_values, y_values = convertToNumpy(fileNames[:NumberOfFileToBetested])

# print x_values
# print y_values

In [4]:
flattened = [ np.ravel(matrices) for matrices in x_values ]

In [16]:
flat_rdd = sc.parallelize(flattened).map(lambda x : x.tolist()).map(lambda x: [int(element) for element in x])
len(flat_rdd.take(5)[0])

49152

In [7]:
age_rdd = sc.parallelize(y_values).map(lambda x:int(x))
# age_rdd.take(5)

In [17]:
combined = flat_rdd.zip(age_rdd)
# combined.collect()[0][1]

In [20]:
#Create a DataFrame
from pyspark.sql.types import *
from pyspark.sql import Row
penschema = StructType([
   StructField("features", ArrayType(elementType=IntegerType(),containsNull=False),True),
   StructField("label", IntegerType(),True)
])
dfpen1 = sqlContext.createDataFrame(combined.map(lambda x : Row(x[0][:],x[1])), penschema)



In [24]:
dfpen1.

TypeError: 'Column' object is not callable

In [11]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
dfpen1 = dfpen1.select(list_to_vector_udf(dfpen1["features"]).alias("features"),'label')

pendtsets = dfpen1.randomSplit([0.8, 0.2])
pendttrain = pendtsets[0].cache()
pendtvalid = pendtsets[1].cache()

In [12]:
# pendttrain.printSchema()

In [13]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(regParam=0.01, maxIter=10, fitIntercept=True)
lrmodel = lr.fit(pendttrain)

Py4JJavaError: An error occurred while calling o102.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 1 times, most recent failure: Lost task 1.0 in stage 2.0 (TID 6, localhost, executor driver): java.lang.IllegalArgumentException: requirement failed: Dimensions mismatch when adding new sample. Expecting 49152 but got 16384.
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.mllib.stat.MultivariateOnlineSummarizer.add(MultivariateOnlineSummarizer.scala:87)
	at org.apache.spark.ml.classification.LogisticRegression$$anonfun$15.apply(LogisticRegression.scala:509)
	at org.apache.spark.ml.classification.LogisticRegression$$anonfun$15.apply(LogisticRegression.scala:508)
	at scala.collection.TraversableOnce$$anonfun$foldLeft$1.apply(TraversableOnce.scala:157)
	at scala.collection.TraversableOnce$$anonfun$foldLeft$1.apply(TraversableOnce.scala:157)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at org.apache.spa